# Подбор моделей для этапа генерации кандидатов

### Описание
Для пакетной рекомендательной системы характерны 3 этапа работы:
1. Генерация кандидатов. На этом этапе происходит отбор сотен или тысяч объектов-кандидатов из множества объектов, которых могут быть миллиарды. Цель заключается в том, чтобы включить в этот набор макси-мально релевантные пользователю объекты и исключить нерелевантные.
2. Скоринг и ранжирование. Здесь система оценивает кандидатов, полученных на предыдущем этапе, и присваивает каждому из них баллы, основанные на их релевантности для пользователя. Т.к. количество объек-тов на этом этапе ограничено, то можно использовать больший набор при-знаков объектов, контекстные факторы, например время и геопозицию.
3. Повторное ранжирование. В рамках данного этапа проводится корректировка списка кандидатов, применяются бизнес-правила, использу-ются метрики разнообразия и новизны для уточнения позиций объектов-кандидатов в итоговом списке рекомендаций.

### Имеющиеся данные
социально-демографическая информация о пользователях и список их покупок - дата, название, цена, количество.

Разделение на трейн и тест: даты покупок. Самые последние покупки в тесте.
Названия датафреймов с данными: train_data и test_data.

**Состав train_data и test_data**:
```
 0   buyer_id                           object  идентификатор покупателя
 1   buyer_birth_date                   object  дата рождения покупателя
 2   buyer_sms_allowed                  int64   покупатель разрешил присылать СМС
 3   buyer_emails_allowed               int64   покупатель разрешил присылать email
 4   buyer_account_status_loyal         int64   покупатель в статусе "Лояльный" (0 или 1)
 5   buyer_account_status_unregistered  int64   покупатель в статусе "Не зарегистрирован" (0 или 1)
 6   buyer_account_status_new           int64   покупатель в статусе "Новый" (0 или 1)
 7   buyer_account_status_potential     int64   покупатель в статусе "Потенциальный" (0 или 1)
 8   buyer_account_status_lost          int64   покупатель в статусе "Потерянный" (0 или 1)
 9   buyer_account_status_sleeping      int64   покупатель в статусе "Спящий" (0 или 1)
 10  buyer_is_female                    int64   покупатель - женщина (0 или 1)
 11  buyer_age                          int64   возраст покупателя
 12  order_id                           object  идентификатор покупки
 13  order_date                         object  дата покупки
 14  order_day_of_week                  int64   день недели, в который была совершена покупка
 15  order_hour_of_day                  int64   час дня, в который была совершена покупка
 16  product_id                         object  идентификатор товара
 17  product_name                       object  название товара
 18  product_group                      object  название группы товара
 19  product_count                      float64 количество товара
 20  product_sum                        float64 сумма за товар
```

### Метрики
- HitRate@K
- Precision@K
- Recall@K
- Diversity@K
- Novelty@K
- Serendipity@K

Оптимизируем метрику Precision@K, т.к. на этапе генерации кандидатов основной задачей сформировать релевантный пользователю пулл кандидатов, их последовательность не важна.

## Импорты и конфигурация

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import scipy.sparse as sparse
from implicit.cpu.als import AlternatingLeastSquares
import optuna
from collections import defaultdict, Counter
from sklearn.preprocessing import StandardScaler
from typing import Dict, List, Set, Tuple, Optional, Any
import os
import multiprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib
from scipy.sparse import csr_matrix
import logging
import warnings
from lightfm import LightFM
from concurrent.futures import ProcessPoolExecutor
import json

/home/jupyter/.local/lib/python3.10/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [2]:
ORGANIZATION_ID = ""
PROCESSING_DATE = ""
DATA_PATH = ""


In [3]:
warnings.filterwarnings("ignore")
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True
)

## Классы-утилиты и общие методы

In [27]:
class MetricsCalculator:
    """
    Калькулятор метрик
    """

    def __init__(self, k_values: List[int]):
        self.k_values = sorted(k_values)
        self.max_k = max(k_values)
        self._total_categories_count = None
        self._item_novelty_scores = None
        self._primitive_model_recs = None

    def _get_primitive_recommendations(
        self, train_data: pd.DataFrame, n_items: int
    ) -> List[str]:
        """
        Получает список рекомендаций от примитивной модели (на основе популярности)

        Args:
            train_data: тренировочный датафрейм
            n_items: количество товаров для рекомендации

        Returns:
            List[str]: список ID товаров, отсортированный по популярности
        """
        # Считаем популярность товаров
        item_popularity = train_data["product_id"].value_counts()
        # Берем top-n самых популярных товаров
        return item_popularity.head(n_items).index.tolist()

    def _calculate_novelty_scores(self, train_data: pd.DataFrame) -> Dict[str, float]:
        """
        Рассчитывает нормализованную новизну для каждого товара на основе частоты его появления
        в тренировочных данных.

        Normalized_Novelty(i) = -log2(|U_i| / |U|) / log2(|U|), где
        |U_i| - количество пользователей, взаимодействовавших с товаром i
        |U| - общее количество пользователей

        Returns:
            Dict[str, float]: словарь {item_id: normalized_novelty_score}
        """
        # Подсчитываем количество пользователей для каждого товара
        item_users = Counter(train_data["product_id"])
        total_users = len(train_data["buyer_id"].unique())
        log2_total_users = np.log2(total_users)

        # Рассчитываем нормализованную новизну для каждого товара
        novelty_scores = {}
        for item, count in item_users.items():
            # Нормализуем на log2(|U|), чтобы получить значения в диапазоне [0, 1]
            novelty_scores[item] = -np.log2(count / total_users) / log2_total_users

        return novelty_scores

    def _calculate_user_metrics(
        self,
        user_recommendations: List[str],
        user_true_items: Set[str],
        item_categories: Optional[Dict[str, str]] = None,
    ) -> Dict[str, float]:
        """
        Расчет метрик для одного пользователя
        """
        metrics = defaultdict(float)

        if not user_true_items:  # пропускаем пользователей без покупок в тесте
            return metrics

        # Предварительно получаем категории для всех рекомендаций до max_k
        if item_categories is not None:
            rec_categories_list = [
                item_categories.get(item)
                for item in user_recommendations[: self.max_k]
                if item in item_categories
            ]

        # Для каждого значения K
        for k in self.k_values:
            rec_items = set(user_recommendations[:k])
            hits = rec_items & user_true_items

            # Hit Rate @K
            metrics[f"hit_rate_{k}"] = float(len(hits) > 0)

            # Precision @K
            metrics[f"precision_{k}"] = len(hits) / k

            # Recall @K
            metrics[f"recall_{k}"] = len(hits) / len(user_true_items)

            # Diversity @K
            if item_categories is not None:
                unique_categories = len(set(rec_categories_list[:k]))
                metrics[f"diversity_{k}"] = (
                    unique_categories / self._total_categories_count
                )

            # Novelty @K
            if self._item_novelty_scores is not None:
                novelty_sum = sum(
                    self._item_novelty_scores.get(item, 0.0)
                    for item in user_recommendations[:k]
                )
                metrics[f"novelty_{k}"] = novelty_sum / k if k > 0 else 0.0

            # Serendipity @K
            if self._primitive_model_recs is not None:
                primitive_recs = set(self._primitive_model_recs[:k])
                unexpected_hits = hits - primitive_recs
                metrics[f"serendipity_{k}"] = len(unexpected_hits) / k if k > 0 else 0.0

        return metrics

    def calculate(
        self,
        recommendations: Dict[str, List[str]],
        train_data: pd.DataFrame,
        test_data: pd.DataFrame,
        item_categories: Optional[Dict[str, str]] = None,
    ) -> Dict[str, float]:
        """
        Расчет всех метрик

        Args:
            recommendations: словарь {user_id: list of recommended items}
            train_data: тренировочный датафрейм для расчета новизны
            test_data: тестовый датафрейм с колонками 'buyer_id' и 'product_id'
            item_categories: словарь {item_id: category_id} для расчета разнообразия

        Returns:
            Dict[str, float]: словарь с метриками:
                - hit_rate_{k}: доля пользователей, у которых хотя бы один товар угадан в топ-k
                - precision_{k}: доля релевантных товаров среди k рекомендованных
                - recall_{k}: доля угаданных товаров от всех релевантных
                - diversity_{k}: среднее разнообразие категорий в топ-k рекомендациях
                - novelty_{k}: средняя новизна товаров в топ-k рекомендациях
                - serendipity_{k}: доля релевантных товаров, не входящих в топ популярных
                где k принимает значения из self.k_values
        """
        # Подготовка данных о реальных покупках
        true_items = test_data.groupby("buyer_id")["product_id"].agg(set).to_dict()

        # Инициализация счетчиков метрик
        total_metrics = defaultdict(float)

        # Предварительный расчет количества всех категорий
        if item_categories is not None:
            self._total_categories_count = len(set(item_categories.values()))

        # Расчет новизны товаров
        self._item_novelty_scores = self._calculate_novelty_scores(train_data)

        # Получаем рекомендации примитивной модели для расчета serendipity
        self._primitive_model_recs = self._get_primitive_recommendations(
            train_data, max(self.k_values)
        )

        # Расчет метрик для каждого пользователя
        valid_users = set(true_items.keys()) & set(recommendations.keys())

        for user_id in valid_users:
            user_metrics = self._calculate_user_metrics(
                recommendations[user_id], true_items[user_id], item_categories
            )
            # Суммируем метрики пользователя
            for metric, value in user_metrics.items():
                total_metrics[metric] += value

        # Нормализация метрик
        total_users = len(valid_users)
        final_metrics = {
            metric: value / total_users for metric, value in total_metrics.items()
        }

        recommended_items_by_k = defaultdict(set)

        # Группируем рекомендации по K за один проход
        for recs in recommendations.values():
            for k in self.k_values:
                recommended_items_by_k[k].update(recs[:k])

        return final_metrics

In [5]:
def save_optimization_results(
    params: Dict[str, Any],
    metrics: Dict[str, float],
    model_type: str = "als",
) -> str:
    """
    Сохраняет результаты оптимизации (параметры и метрики) в JSON файл.

    Args:
        params: словарь с лучшими параметрами
        metrics: словарь с метриками
        organization_id: ID организации
        model_type: тип модели (als/hybrid)
        base_path: базовый путь для сохранения

    Returns:
        str: путь к сохраненному файлу
    """
    save_path = os.path.expanduser(
        f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_{model_type}_optimization_results.json"
    )

    # Подготавливаем данные для сохранения
    results = {
        "best_parameters": params,
        "best_metrics": metrics,
        "optimization_info": {
            "organization_id": ORGANIZATION_ID,
            "model_type": model_type,
            "pricessing_date": PROCESSING_DATE,
        },
    }

    # Сохраняем в JSON
    try:
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
    except Exception as e:
        logging.error(f"Ошибка при сохранении результатов: {str(e)}")
        raise

## Данные

In [6]:
buyers_df = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_buyers.csv")

In [7]:
buyers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247074 entries, 0 to 247073
Data columns (total 12 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   buyer_id                           247074 non-null  object
 1   birth_date                         247074 non-null  object
 2   sms_allowed                        247074 non-null  int64 
 3   emails_allowed                     247074 non-null  int64 
 4   account_status_Лояльный            247074 non-null  int64 
 5   account_status_Не зарегистрирован  247074 non-null  int64 
 6   account_status_Новый               247074 non-null  int64 
 7   account_status_Потенциальный       247074 non-null  int64 
 8   account_status_Потерянный          247074 non-null  int64 
 9   account_status_Спящий              247074 non-null  int64 
 10  is_female                          247074 non-null  int64 
 11  age                                247074 non-null  

In [8]:
buyers_df.sample(3)

,buyer_id,birth_date,sms_allowed,emails_allowed,account_status_Лояльный,account_status_Не зарегистрирован,account_status_Новый,account_status_Потенциальный,account_status_Потерянный,account_status_Спящий,is_female,age
208522,0feab126-7533-232c-3f16-8673113c8daa,2002-02-02 00:00:00.000000000,1,1,0,0,0,0,1,0,1,23
23142,7c88aa1c-7a85-f5fb-86c9-ce19e3836cd7,1969-07-08 00:00:00.000000000,1,0,1,0,0,0,0,0,1,56
104555,fc96e091-b230-6b9f-f029-62a95bdc5c78,1974-10-31 03:41:22.541633280,1,1,0,0,0,0,1,0,1,51


In [9]:
products_df = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_products.csv")

In [10]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24858 entries, 0 to 24857
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     24858 non-null  object
 1   product_name   24858 non-null  object
 2   product_group  24858 non-null  object
dtypes: object(3)
memory usage: 582.7+ KB


In [11]:
products_df.sample(3)

,product_id,product_name,product_group
13036,e2d5fa8e-45eb-1c90-245e-e5da7e86ba36,"Вода б/г 0,5",напитки
22071,d198b925-bc8b-1b17-b230-897335a65a15,Сыр Пиканта с вялеными томатами оливками и коп...,сыр
14431,a02732f1-396a-9852-f4af-613b5de3550a,Мука Пшеничная Хлебопекарная В/С 2 Кг Агрокомп...,агрокомплекс


In [12]:
orders_df = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_orders.csv")

In [13]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12398107 entries, 0 to 12398106
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   buyer_id       object 
 1   order_id       object 
 2   order_date     object 
 3   product_id     object 
 4   product_count  float64
 5   product_sum    float64
 6   day_of_week    int64  
 7   hour           int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 756.7+ MB


In [14]:
orders_df.sample(3)

,buyer_id,order_id,order_date,product_id,product_count,product_sum,day_of_week,hour
2566979,d0c29a73-5591-1baf-0ba6-4b6ef6fa6e27,76fcb970-38ff-d8dc-0ba2-09ab9abdd9ce,2024-05-27 12:32:04,292a10a1-be49-adcc-8061-79414ecec05e,1.000,25.00,1,12
1298138,a49bfa6d-c563-d4eb-e715-5000a95cfd5f,76e20bb2-1aff-7d22-378d-fb2ad510c304,2024-04-22 18:16:26,f72080ea-66c7-1c6f-6c90-16159c1f9a93,0.534,325.21,1,18
8108364,0b381760-cbc0-5a81-568d-eeaa58784200,bb6aed81-b3a7-b6ae-7a11-a73563251e24,2024-10-25 18:32:15,68ecddd1-0c5d-fd93-a506-fcc46c4a5cea,0.802,550.17,5,18


In [15]:
full_df = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_full.csv")


In [16]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12398107 entries, 0 to 12398106
Data columns (total 21 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   buyer_id                           object 
 1   buyer_birth_date                   object 
 2   buyer_sms_allowed                  int64  
 3   buyer_emails_allowed               int64  
 4   buyer_account_status_loyal         int64  
 5   buyer_account_status_unregistered  int64  
 6   buyer_account_status_new           int64  
 7   buyer_account_status_potential     int64  
 8   buyer_account_status_lost          int64  
 9   buyer_account_status_sleeping      int64  
 10  buyer_is_female                    int64  
 11  buyer_age                          int64  
 12  order_id                           object 
 13  order_date                         object 
 14  order_day_of_week                  int64  
 15  order_hour_of_day                  int64  
 16  product_id      

In [17]:
full_df.sample(3)

,buyer_id,buyer_birth_date,buyer_sms_allowed,buyer_emails_allowed,buyer_account_status_loyal,buyer_account_status_unregistered,buyer_account_status_new,buyer_account_status_potential,buyer_account_status_lost,buyer_account_status_sleeping,buyer_is_female,buyer_age,order_id,order_date,order_day_of_week,order_hour_of_day,product_id,product_name,product_group,product_count,product_sum
8795070,58155741-7f0e-43cc-4977-a55261a7127c,1988-06-01 00:00:00.000000000,1,1,1,0,0,0,0,0,1,37,4013abc2-bc04-b947-9e2c-ae162a5a28fa,2024-11-15 16:31:08,5,16,5f4b299d-e2bb-95b0-49ec-25555a119b9f,"Батон ""Нарезной улучшенный"" нарезка 350гр Эл...",батон,1.0,64.0
3104125,7313d58d-5e80-ab26-edbc-33ce87968a8c,1973-05-30 02:43:21.014589280,1,1,1,0,0,0,0,0,1,52,6d9997a7-c314-4b9b-c06e-89b2f24cd90b,2024-06-10 10:09:56,1,10,fff15622-2b1a-a579-9393-f718b1c3877a,"Каппучино LaFesta 12,5 гр.",каппучино,1.0,16.0
9594578,7d7850c9-2e03-ec6d-875d-a55f58b67411,1974-04-29 00:00:00.000000000,1,1,1,0,0,0,0,0,1,51,bdcbdff1-0d7c-b643-9465-61c9b8389f9d,2024-12-10 17:46:53,2,17,d0a14063-51dc-b1d9-87ce-86f8652fc6e1,"Молоко цел.отб.пит.пастер. с м.д.ж от 3,4% до ...",агрокомплекс,1.0,157.0


## Разделение данных

In [18]:
df = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_full.csv")
df["order_date"] = pd.to_datetime(df["order_date"])

In [19]:
# Сокращаем датасет до 2025 года
df = df[df['order_date'] >= datetime.strptime('2025-01-01', '%Y-%m-%d')].copy().reset_index(drop=True)
df.shape

(2014738, 21)

In [20]:
def split_data_for_recsys(
    df: pd.DataFrame, test_days: int = 7, min_user_purchases: int = 2
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Разделение данных на train и test с учетом временных особенностей.

    Args:
        df: DataFrame с данными
        test_days: количество дней для тестового периода
        min_user_purchases: минимальное количество покупок пользователя

    Returns:
        tuple: (train_df, test_df)
    """
    # Конвертация даты в datetime
    df["order_date"] = pd.to_datetime(df["order_date"])

    # Определение временных границ
    max_date = df["order_date"].max()
    test_start_date = max_date - timedelta(days=test_days)

    logging.info(f"Максимальная дата в данных: {max_date.date()}")
    logging.info(f"Дата начала тестового периода: {test_start_date.date()}")

    # Подсчет количества покупок для каждого пользователя
    user_purchase_counts = df.groupby("buyer_id").size()

    # Отбор пользователей с достаточным количеством покупок
    valid_users = user_purchase_counts[user_purchase_counts >= min_user_purchases].index

    # Фильтрация данных по валидным пользователям
    df_filtered = df[df["buyer_id"].isin(valid_users)]

    # Разделение на train и test
    train_df = df_filtered[df_filtered["order_date"] < test_start_date]
    test_df = df_filtered[df_filtered["order_date"] >= test_start_date]

    # Проверка, что у каждого пользователя в тесте есть история в трейне
    test_users = set(test_df["buyer_id"].unique())
    train_users = set(train_df["buyer_id"].unique())
    users_without_history = test_users - train_users

    if users_without_history:
        logging.warning(
            f"Найдено {len(users_without_history)} пользователей в тесте без истории покупок"
        )
        test_df = test_df[~test_df["buyer_id"].isin(users_without_history)]

    # Вывод статистики
    logging.info("\nСтатистика разделения данных:")
    logging.info(f"Всего записей: {len(df)}")
    logging.info(f"Записей после фильтрации: {len(df_filtered)}")
    logging.info(f"Количество записей в train: {len(train_df)}")
    logging.info(f"Количество записей в test: {len(test_df)}")
    logging.info(
        f"Количество уникальных пользователей в train: {train_df['buyer_id'].nunique()}"
    )
    logging.info(
        f"Количество уникальных пользователей в test: {test_df['buyer_id'].nunique()}"
    )
    logging.info(
        f"Количество уникальных товаров в train: {train_df['product_id'].nunique()}"
    )
    logging.info(
        f"Количество уникальных товаров в test: {test_df['product_id'].nunique()}"
    )

    # Проверка временных периодов
    logging.info("\nВременные периоды:")
    logging.info(
        f"Train: с {train_df['order_date'].min().date()} по {train_df['order_date'].max().date()}"
    )
    logging.info(
        f"Test: с {test_df['order_date'].min().date()} по {test_df['order_date'].max().date()}"
    )

    return train_df, test_df

In [21]:
train_data, test_data = split_data_for_recsys(
    df,
    test_days=14,
    min_user_purchases=2,
)
train_data.shape[0], test_data.shape[0]

2025-04-21 09:33:00 - INFO - Максимальная дата в данных: 2025-03-11
2025-04-21 09:33:00 - INFO - Дата начала тестового периода: 2025-02-25
2025-04-21 09:33:01 - WARNING - Найдено 7226 пользователей в тесте без истории покупок
2025-04-21 09:33:01 - INFO - 
Статистика разделения данных:
2025-04-21 09:33:01 - INFO - Всего записей: 2014738
2025-04-21 09:33:01 - INFO - Записей после фильтрации: 2000550
2025-04-21 09:33:01 - INFO - Количество записей в train: 1594743
2025-04-21 09:33:01 - INFO - Количество записей в test: 374993
2025-04-21 09:33:02 - INFO - Количество уникальных пользователей в train: 114454
2025-04-21 09:33:02 - INFO - Количество уникальных пользователей в test: 60050
2025-04-21 09:33:02 - INFO - Количество уникальных товаров в train: 15053
2025-04-21 09:33:02 - INFO - Количество уникальных товаров в test: 10629
2025-04-21 09:33:02 - INFO - 
Временные периоды:
2025-04-21 09:33:02 - INFO - Train: с 2025-01-01 по 2025-02-25
2025-04-21 09:33:02 - INFO - Test: с 2025-02-26 по 2

(1594743, 374993)

In [22]:
train_data.to_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_train.csv", index=False)
test_data.to_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_test.csv", index=False)


## Бейзлайн

In [23]:
train_data = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_train.csv")
test_data = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_test.csv")

# Создаем словарь категорий товаров
item_categories = {
    row['product_id']: row['product_group'] 
    for _, row in train_data.iterrows()
}

In [24]:
class PopularItemsRecommender:
    """
    Простой рекомендатель на основе популярных товаров
    """

    def __init__(self):
        self.popular_items = None

    def fit(self, df: pd.DataFrame, weight_col: str = None) -> None:
        """
        Обучение модели - подсчет популярности товаров

        Args:
            df: DataFrame с историей покупок
            weight_col: колонка для взвешивания (например, 'product_count' или 'product_sum')
        """
        if weight_col:
            # Взвешенная популярность
            item_popularity = df.groupby("product_id")[weight_col].sum()
        else:
            # Простой подсчет количества покупок
            item_popularity = df.groupby("product_id").size()

        # Сортировка по убыванию популярности
        self.popular_items = item_popularity.sort_values(ascending=False).index.tolist()

    def recommend(self, k: int) -> List[str]:
        """
        Получение топ-K популярных товаров

        Args:
            k: количество товаров для рекомендации

        Returns:
            Список идентификаторов товаров
        """
        return self.popular_items[:k]

In [28]:
def evaluate_popular_items_baseline(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    k_values: List[int] = [10, 100, 1000],
    weight_cols: List[str] = [None, "product_count", "product_sum"],
) -> None:
    """
    Оценка бейзлайна на основе популярных товаров с разными вариантами взвешивания.
    Для каждого способа подсчета популярности (количество покупок, количество единиц товара,
    сумма покупок) вычисляются метрики качества рекомендаций.

    Args:
        train_data (pd.DataFrame): Датафрейм с обучающими данными.
            Обязательные колонки: 'buyer_id', 'product_id'.
            Опциональные колонки: 'product_count', 'product_sum'.
        test_data (pd.DataFrame): Датафрейм с тестовыми данными.
            Обязательные колонки: 'buyer_id', 'product_id'.
        k_values (List[int], optional): Список значений K для расчета метрик @K.
            По умолчанию [10, 100, 1000].
        weight_cols (List[str], optional): Список колонок для взвешивания популярности товаров.
            None - подсчет количества покупок,
            'product_count' - взвешивание по количеству единиц товара,
            'product_sum' - взвешивание по сумме покупок.
            По умолчанию [None, 'product_count', 'product_sum'].
    """
    metrics_calculator = MetricsCalculator(k_values)

    for weight_col in weight_cols:
        weight_name = weight_col if weight_col else "count"
        logging.info(f"\nОценка популярности на основе {weight_name}")

        # Обучение рекомендателя
        recommender = PopularItemsRecommender()
        recommender.fit(train_data, weight_col)

        # Генерация рекомендаций для всех пользователей
        test_users = test_data["buyer_id"].unique()
        recommendations = {
            user_id: recommender.recommend(max(k_values)) for user_id in test_users
        }

        # Расчет метрик
        metrics = metrics_calculator.calculate(
            recommendations=recommendations,
            train_data=train_data,
            test_data=test_data,
            item_categories=item_categories,
        )

        # Вывод результатов
        logging.info("Результаты:")
        for k in k_values:
            logging.info(f"Метрики для K={k}:")
            logging.info(f"Hit Rate@{k}: {metrics[f'hit_rate_{k}']:.4f}")
            logging.info(f"Precision@{k}: {metrics[f'precision_{k}']:.4f}")
            logging.info(f"Recall@{k}: {metrics[f'recall_{k}']:.4f}")
            logging.info(f"Diversity@{k}: {metrics[f'diversity_{k}']:.4f}")
            logging.info(f"Novelty@{k}: {metrics[f'novelty_{k}']:.4f}")
            logging.info(f"Serendipity@{k}: {metrics[f'serendipity_{k}']:.4f}")
            logging.info("--------------------------------")

In [29]:
evaluate_popular_items_baseline(train_data, test_data, weight_cols=[None])

2025-04-21 09:54:26 - INFO - 
Оценка популярности на основе count
2025-04-21 09:54:48 - INFO - Результаты:
2025-04-21 09:54:48 - INFO - Метрики для K=10:
2025-04-21 09:54:48 - INFO - Hit Rate@10: 0.3884
2025-04-21 09:54:48 - INFO - Precision@10: 0.0552
2025-04-21 09:54:48 - INFO - Recall@10: 0.1274
2025-04-21 09:54:48 - INFO - Diversity@10: 0.0047
2025-04-21 09:54:48 - INFO - Novelty@10: 0.1611
2025-04-21 09:54:48 - INFO - Serendipity@10: 0.0000
2025-04-21 09:54:48 - INFO - --------------------------------
2025-04-21 09:54:48 - INFO - Метрики для K=100:
2025-04-21 09:54:48 - INFO - Hit Rate@100: 0.7865
2025-04-21 09:54:48 - INFO - Precision@100: 0.0190
2025-04-21 09:54:48 - INFO - Recall@100: 0.4382
2025-04-21 09:54:48 - INFO - Diversity@100: 0.0313
2025-04-21 09:54:48 - INFO - Novelty@100: 0.2698
2025-04-21 09:54:48 - INFO - Serendipity@100: 0.0000
2025-04-21 09:54:48 - INFO - --------------------------------
2025-04-21 09:54:48 - INFO - Метрики для K=1000:
2025-04-21 09:54:48 - INFO 

## Модель 1: Alternating Least Squares (Коллаборативная фильтрация)

In [30]:
train_data = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_train.csv")
test_data = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_test.csv")

item_categories = {
    row['product_id']: row['product_group'] 
    for _, row in train_data.iterrows()
}

In [31]:
class ALSRecommender:
    """
    Рекомендательная система на основе Alternating Least Squares
    """

    def __init__(
        self,
        user_col: str = "buyer_id",
        item_col: str = "product_id",
        weight_col: str = "product_count",
        random_state: int = 42,
    ):
        """
        Args:
            user_col: Название колонки с ID пользователей
            item_col: Название колонки с ID товаров
            weight_col: Название колонки для весов взаимодействий
            random_state: Фиксация случайности
        """
        self.user_col = user_col
        self.item_col = item_col
        self.weight_col = weight_col
        self.random_state = random_state
        self.model = None

    def fit(self, train_data: pd.DataFrame, model_params: Dict) -> None:
        """
        Обучение модели

        Args:
            train_data: Обучающий датафрейм
            model_params: Параметры модели ALS
        """
        self.user_mapping = {
            uid: idx for idx, uid in enumerate(train_data[self.user_col].unique())
        }
        self.item_mapping = {
            iid: idx for idx, iid in enumerate(train_data[self.item_col].unique())
        }
        self.reverse_user_mapping = {v: k for k, v in self.user_mapping.items()}
        self.reverse_item_mapping = {v: k for k, v in self.item_mapping.items()}

        self.interactions_sparse = self._create_sparse_matrix(train_data)

        self.model = AlternatingLeastSquares(
            factors=model_params["factors"],
            regularization=model_params["regularization"],
            alpha=model_params["alpha"],
            iterations=model_params["iterations"],
            random_state=self.random_state,
        )
        self.model.fit(self.interactions_sparse)

    def _create_sparse_matrix(self, data: pd.DataFrame) -> csr_matrix:
        """
        Создание разреженной матрицы взаимодействий пользователь-товар.

        Преобразует датафрейм с историей взаимодействий в разреженную матрицу формата CSR,
        где строки соответствуют пользователям, столбцы - товарам, а значения - весам взаимодействий.

        Args:
            data (pd.DataFrame): Датафрейм с данными о взаимодействиях.
                Должен содержать колонки:
                - self.user_col (str): колонка с ID пользователей
                - self.item_col (str): колонка с ID товаров
                - self.weight_col (str, optional): колонка с весами взаимодействий
                                                (например, количество товара или сумма покупки)

        Returns:
            csr_matrix: Разреженная матрица взаимодействий размера (n_users, n_items), где:
                - n_users: количество уникальных пользователей
                - n_items: количество уникальных товаров
                Значения в матрице:
                - если weight_col задан: веса из указанной колонки
                - если weight_col не задан: 1 для каждого взаимодействия

        Note:
            Метод использует предварительно созданные маппинги self.user_mapping и self.item_mapping
            для преобразования ID в индексы матрицы.
        """
        user_indices = [self.user_mapping[uid] for uid in data[self.user_col]]
        item_indices = [self.item_mapping[iid] for iid in data[self.item_col]]

        weights = (
            data[self.weight_col].values if self.weight_col else np.ones(len(data))
        )

        return csr_matrix(
            (weights, (user_indices, item_indices)),
            shape=(len(self.user_mapping), len(self.item_mapping)),
        )

    def recommend(
        self, users: List[str], k: int, filter_already_liked: bool = False
    ) -> Dict[str, List[str]]:
        """
        Генерация рекомендаций для списка пользователей

        Args:
            users: Список ID пользователей
            k: Количество рекомендаций для каждого пользователя
            filter_already_liked: Исключать ли уже купленные товары

        Returns:
            Dict[str, List[str]]: Словарь {user_id: [product_id_1, product_id_2, ...]}
        """
        if self.model is None:
            raise ValueError("Модель не обучена. Сначала выполните fit().")

        recommendations = {}

        for user_id in users:
            if user_id in self.user_mapping:
                user_idx = self.user_mapping[user_id]
                indices, _ = self.model.recommend(
                    user_idx,
                    self.interactions_sparse[user_idx],
                    N=k,
                    filter_already_liked_items=filter_already_liked,
                )
                recommendations[user_id] = [
                    self.reverse_item_mapping[idx] for idx in indices
                ]
            else:
                recommendations[user_id] = []

        return recommendations
    
    def save_model(self, path: str) -> None:
        """
        Сохранение модели и всех необходимых атрибутов в файл используя joblib

        Args:
            path: Путь для сохранения модели
        """
        if self.model is None:
            raise ValueError("Модель не обучена. Сначала выполните fit().")
        
        model_data = {
            'model': self.model,
            'user_mapping': self.user_mapping,
            'item_mapping': self.item_mapping,
            'reverse_user_mapping': self.reverse_user_mapping,
            'reverse_item_mapping': self.reverse_item_mapping,
            'interactions_sparse': self.interactions_sparse,
            'user_col': self.user_col,
            'item_col': self.item_col,
            'weight_col': self.weight_col,
            'random_state': self.random_state
        }
        
        joblib.dump(model_data, path, compress=3)

    @classmethod
    def load_model(cls, path: str) -> "ALSRecommender":
        """
        Загрузка модели из файла используя joblib

        Args:
            path: Путь к сохраненной модели

        Returns:
            ALSRecommender: Загруженная модель с восстановленными атрибутами
        """
        model_data = joblib.load(path)
        
        # Создаем новый экземпляр класса
        recommender = cls(
            user_col=model_data['user_col'],
            item_col=model_data['item_col'],
            weight_col=model_data['weight_col'],
            random_state=model_data['random_state']
        )
        
        recommender.model = model_data['model']
        recommender.user_mapping = model_data['user_mapping']
        recommender.item_mapping = model_data['item_mapping']
        recommender.reverse_user_mapping = model_data['reverse_user_mapping']
        recommender.reverse_item_mapping = model_data['reverse_item_mapping']
        recommender.interactions_sparse = model_data['interactions_sparse']
        
        return recommender

In [32]:
def optimize_als_parameters(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    metrics_calculator: MetricsCalculator,
    n_trials: int = 50,
    timeout: int = 3600,
) -> Tuple[Dict, Dict[str, float]]:
    """
    Оптимизация параметров ALS с помощью Optuna

    Args:
        train_data: Обучающий датафрейм
        test_data: Тестовый датафрейм
        metrics_calculator: Инициализированный калькулятор метрик
        n_trials: Количество итераций оптимизации
        timeout: Максимальное время оптимизации в секундах

    Returns:
        Tuple[Dict, Dict[str, float]]: (лучшие параметры, метрики на лучших параметрах)
    """

    def objective(trial: optuna.Trial) -> float:
        params = {
            "factors": trial.suggest_int("factors", 32, 256, step=32),
            "regularization": trial.suggest_float(
                "regularization", 0.01, 1.0, log=True
            ),
            "alpha": trial.suggest_float("alpha", 0.1, 40.0, log=True),
            "iterations": trial.suggest_int("iterations", 10, 50),
        }

        recommender = ALSRecommender(weight_col="product_count")
        recommender.fit(train_data, params)

        test_users = test_data["buyer_id"].unique()
        recommendations = recommender.recommend(
            users=test_users, k=max(metrics_calculator.k_values)
        )

        metrics = metrics_calculator.calculate(
            recommendations=recommendations,
            train_data=train_data,
            test_data=test_data,
            item_categories=item_categories,
        )

        target_k = max(metrics_calculator.k_values)
        return metrics[f"recall_{target_k}"]

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, timeout=timeout)

    best_params = study.best_params

    best_recommender = ALSRecommender(weight_col="product_count")
    best_recommender.fit(train_data, best_params)

    test_users = test_data["buyer_id"].unique()
    final_recommendations = best_recommender.recommend(
        users=test_users, k=max(metrics_calculator.k_values)
    )

    final_metrics = metrics_calculator.calculate(
        recommendations=final_recommendations,
        train_data=train_data,
        test_data=test_data,
        item_categories=item_categories,
    )

    return best_params, final_metrics

In [33]:
metrics_calculator = MetricsCalculator([10, 100, 1000])

best_params, best_metrics = optimize_als_parameters(
    train_data=train_data,
    test_data=test_data,
    metrics_calculator=metrics_calculator,
    timeout=7200,
)

logging.info(f"Лучшие параметры: {best_params}")
logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"Hit Rate@{k}: {best_metrics[f'hit_rate_{k}']:.4f}")
    logging.info(f"Precision@{k}: {best_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {best_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {best_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {best_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {best_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")

[I 2025-04-21 09:56:02,823] A new study created in memory with name: no-name-ddf20fb0-4c5b-4a09-8978-2c7f6606199c
100%|██████████| 26/26 [00:34<00:00,  1.34s/it]
[I 2025-04-21 10:00:16,207] Trial 0 finished with value: 0.7962044843931364 and parameters: {'factors': 32, 'regularization': 0.030439592843596593, 'alpha': 13.4713502084299, 'iterations': 26}. Best is trial 0 with value: 0.7962044843931364.
100%|██████████| 40/40 [01:26<00:00,  2.17s/it]
[I 2025-04-21 10:05:27,601] Trial 1 finished with value: 0.7801036018081446 and parameters: {'factors': 96, 'regularization': 0.5901031586326936, 'alpha': 27.85971627047329, 'iterations': 40}. Best is trial 0 with value: 0.7962044843931364.
100%|██████████| 44/44 [00:56<00:00,  1.29s/it]
[I 2025-04-21 10:10:01,772] Trial 2 finished with value: 0.7670624762912543 and parameters: {'factors': 32, 'regularization': 0.10448848268937484, 'alpha': 2.855943881341473, 'iterations': 44}. Best is trial 0 with value: 0.7962044843931364.
100%|██████████| 

In [34]:
save_optimization_results(params=best_params, metrics=best_metrics, model_type="als")

## Модель 2: Content-Based (Контентная фильтрация)

In [35]:
train_data = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_train.csv")
test_data = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_test.csv")

item_categories = {
    row['product_id']: row['product_group'] 
    for _, row in train_data.iterrows()
}

In [36]:
class ContentBasedRecommender:
    """
    Рекомендательная система на основе контента товаров
    """

    def __init__(
        self,
        user_col: str = "buyer_id",
        item_col: str = "product_id",
        text_cols: List[str] = ["product_name", "product_group"],
        weight_col: str = "product_count",
        random_state: int = 42,
    ):
        """
        Args:
            user_col: Название колонки с ID пользователей
            item_col: Название колонки с ID товаров
            text_cols: Список колонок с текстовыми описаниями товаров
            weight_col: Название колонки для весов взаимодействий
            random_state: Фиксация случайности
        """
        self.user_col = user_col
        self.item_col = item_col
        self.text_cols = text_cols
        self.weight_col = weight_col
        self.random_state = random_state
        self.vectorizer = None
        self.item_vectors = None

    def _preprocess_text(self, row: pd.Series) -> str:
        """
        Предобработка текстовых признаков товара.

        Метод объединяет значения всех текстовых колонок товара в одну строку
        и выполняет базовую предобработку текста.

        Args:
            row (pd.Series): Строка датафрейма, содержащая текстовые признаки товара.
                Должна включать все колонки из self.text_cols.

        Returns:
            str: Предобработанная строка, содержащая объединенные текстовые признаки.
                Все символы приведены к нижнему регистру.
                Пустые значения (None, nan) заменяются на пустую строку.

        Example:
            Для row с колонками ['product_name', 'product_group']:
            row['product_name'] = 'Молоко Простоквашино'
            row['product_group'] = 'Молочные продукты'
            Вернет: 'молоко простоквашино молочные продукты'
        """
        text = " ".join(str(row[col]).lower() for col in self.text_cols)
        return text

    def fit(self, train_data: pd.DataFrame, model_params: Dict) -> None:
        """
        Обучение модели

        Args:
            train_data: Обучающий датафрейм
            model_params: Параметры модели:
                - max_features: максимальное количество признаков
                - ngram_range: диапазон n-грамм
                - min_df: минимальная частота встречаемости термов
                - max_df: максимальная частота встречаемости термов
        """
        self.user_mapping = {
            uid: idx for idx, uid in enumerate(train_data[self.user_col].unique())
        }
        self.item_mapping = {
            iid: idx for idx, iid in enumerate(train_data[self.item_col].unique())
        }
        self.reverse_user_mapping = {v: k for k, v in self.user_mapping.items()}
        self.reverse_item_mapping = {v: k for k, v in self.item_mapping.items()}

        unique_items = train_data[[self.item_col] + self.text_cols].drop_duplicates()
        item_descriptions = unique_items.apply(self._preprocess_text, axis=1)

        self.vectorizer = TfidfVectorizer(
            max_features=model_params.get("max_features", 10000),
            ngram_range=model_params.get("ngram_range", (1, 2)),
            min_df=model_params.get("min_df", 2),
            max_df=model_params.get("max_df", 0.95),
        )
        self.item_vectors = self.vectorizer.fit_transform(item_descriptions)

        self.user_profiles = self._create_user_profiles(train_data)

    def _create_user_profiles(self, data: pd.DataFrame) -> Dict[int, csr_matrix]:
        """
        Создание профилей пользователей на основе их покупок.

        Для каждого пользователя создается профиль как взвешенная сумма векторов купленных им товаров.
        Веса определяются либо количеством купленных единиц товара (product_count),
        либо суммой покупки (product_sum), либо равномерно (если weight_col не указан).

        Args:
            data (pd.DataFrame): Датафрейм с историей покупок.
                Должен содержать колонки:
                - self.user_col: ID пользователя
                - self.item_col: ID товара
                - self.weight_col (опционально): веса взаимодействий

        Returns:
            Dict[int, csr_matrix]: Словарь профилей пользователей, где:
                - ключ: индекс пользователя (из user_mapping)
                - значение: разреженная матрица-профиль пользователя размерности (1, n_features),
                        где n_features - размерность векторного представления товаров

        Note:
            - Пользователи без покупок или с покупками неизвестных товаров пропускаются
            - Веса нормализуются для каждого пользователя (сумма весов = 1)
            - Профиль пользователя имеет ту же размерность, что и векторы товаров
        """
        user_profiles = {}

        for user_id, user_items in data.groupby(self.user_col):
            if user_id not in self.user_mapping:
                continue

            item_indices = [
                self.item_mapping[iid]
                for iid in user_items[self.item_col]
                if iid in self.item_mapping
            ]

            if not item_indices:
                continue

            if self.weight_col:
                weights = user_items[self.weight_col].values
                weights = weights / weights.sum()
            else:
                weights = np.ones(len(item_indices)) / len(item_indices)

            user_profile = sum(
                self.item_vectors[idx].multiply(weight)
                for idx, weight in zip(item_indices, weights)
            )

            user_profiles[self.user_mapping[user_id]] = user_profile

        return user_profiles

    def recommend(
        self, users: List[str], k: int, filter_already_liked: bool = False
    ) -> Dict[str, List[str]]:
        """
        Генерация рекомендаций для списка пользователей

        Args:
            users: Список ID пользователей
            k: Количество рекомендаций для каждого пользователя
            filter_already_liked: Исключать ли уже купленные товары

        Returns:
            Dict[str, List[str]]: Словарь {user_id: [product_id_1, product_id_2, ...]}
        """
        if self.vectorizer is None or self.item_vectors is None:
            raise ValueError("Модель не обучена. Сначала выполните fit()")

        recommendations = {}

        for user_id in users:
            if (
                user_id in self.user_mapping
                and self.user_mapping[user_id] in self.user_profiles
            ):
                user_profile = self.user_profiles[self.user_mapping[user_id]]

                similarities = cosine_similarity(user_profile, self.item_vectors)[0]

                if filter_already_liked:
                    already_liked = set(
                        self.item_mapping[iid]
                        for iid in self.train_data[
                            self.train_data[self.user_col] == user_id
                        ][self.item_col]
                    )
                    candidate_indices = [
                        i for i in range(len(similarities)) if i not in already_liked
                    ]
                    similarities = similarities[candidate_indices]
                    top_indices = np.argsort(similarities)[-k:][::-1]
                    top_items = [candidate_indices[idx] for idx in top_indices]
                else:
                    top_items = np.argsort(similarities)[-k:][::-1]

                recommendations[user_id] = [
                    self.reverse_item_mapping[idx] for idx in top_items
                ]
            else:
                recommendations[user_id] = []

        return recommendations

    def save_model(self, path: str) -> None:
        """
        Сохранение модели и всех необходимых атрибутов в файл используя joblib

        Args:
            path: Путь для сохранения модели
        """
        if self.vectorizer is None or self.item_vectors is None:
            raise ValueError("Модель не обучена. Сначала выполните fit()")

        model_data = {
            "vectorizer": self.vectorizer,
            "item_vectors": self.item_vectors,
            "user_profiles": self.user_profiles,
            "user_mapping": self.user_mapping,
            "item_mapping": self.item_mapping,
            "reverse_user_mapping": self.reverse_user_mapping,
            "reverse_item_mapping": self.reverse_item_mapping,
            "user_col": self.user_col,
            "item_col": self.item_col,
            "text_cols": self.text_cols,
            "weight_col": self.weight_col,
            "random_state": self.random_state,
        }

        joblib.dump(model_data, path, compress=3)

    @classmethod
    def load_model(cls, path: str) -> "ContentBasedRecommender":
        """
        Загрузка модели из файла используя joblib

        Args:
            path: Путь к сохраненной модели

        Returns:
            ContentBasedRecommender: Загруженная модель с восстановленными атрибутами
        """
        model_data = joblib.load(path)

        recommender = cls(
            user_col=model_data["user_col"],
            item_col=model_data["item_col"],
            text_cols=model_data["text_cols"],
            weight_col=model_data["weight_col"],
            random_state=model_data["random_state"],
        )

        recommender.vectorizer = model_data["vectorizer"]
        recommender.item_vectors = model_data["item_vectors"]
        recommender.user_profiles = model_data["user_profiles"]
        recommender.user_mapping = model_data["user_mapping"]
        recommender.item_mapping = model_data["item_mapping"]
        recommender.reverse_user_mapping = model_data["reverse_user_mapping"]
        recommender.reverse_item_mapping = model_data["reverse_item_mapping"]

        return recommender

In [37]:
def optimize_content_based_parameters(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    metrics_calculator: MetricsCalculator,
    n_trials: int = 50,
    timeout: int = 3600,
) -> Tuple[Dict, Dict[str, float]]:
    """
    Оптимизация параметров Content-Based рекомендера с помощью Optuna
    """

    def objective(trial: optuna.Trial) -> float:
        params = {
            "max_features": trial.suggest_int("max_features", 1000, 50000, step=24000),
            "ngram_range": (1, trial.suggest_int("max_ngram", 1, 5, step=2)),
            "min_df": trial.suggest_float("min_df", 0.001, 0.1),
            "max_df": trial.suggest_float("max_df", 0.5, 0.95),
        }

        recommender = ContentBasedRecommender(weight_col="product_count")
        recommender.fit(train_data, params)

        test_users = test_data["buyer_id"].unique()
        recommendations = recommender.recommend(
            users=test_users, k=max(metrics_calculator.k_values)
        )

        metrics = metrics_calculator.calculate(
            recommendations=recommendations,
            train_data=train_data,
            test_data=test_data,
            item_categories=item_categories,
        )

        target_k = max(metrics_calculator.k_values)
        return metrics[f"recall_{target_k}"]

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, timeout=timeout)

    best_params = study.best_params

    best_recommender = ContentBasedRecommender(weight_col="product_count")
    best_recommender.fit(train_data, best_params)

    test_users = test_data["buyer_id"].unique()
    final_recommendations = best_recommender.recommend(
        users=test_users, k=max(metrics_calculator.k_values)
    )

    final_metrics = metrics_calculator.calculate(
        recommendations=final_recommendations,
        train_data=train_data,
        test_data=test_data,
        item_categories=item_categories,
    )

    return best_params, final_metrics

In [38]:
metrics_calculator = MetricsCalculator([10, 100, 1000])

best_params, best_metrics = optimize_content_based_parameters(
    train_data=train_data, test_data=test_data, metrics_calculator=metrics_calculator, timeout=7200
)

logging.info(f"Лучшие параметры: {best_params}")
logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"Hit Rate@{k}: {best_metrics[f'hit_rate_{k}']:.4f}")
    logging.info(f"Precision@{k}: {best_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {best_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {best_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {best_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {best_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")

[I 2025-04-21 12:01:19,360] A new study created in memory with name: no-name-181e042a-978f-4095-a2ed-a987af1bb376
[I 2025-04-21 12:07:50,169] Trial 0 finished with value: 0.02426948001027788 and parameters: {'max_features': 49000, 'max_ngram': 1, 'min_df': 0.06930012360020005, 'max_df': 0.5030907587861031}. Best is trial 0 with value: 0.02426948001027788.
[I 2025-04-21 12:14:19,351] Trial 1 finished with value: 0.02426948001027788 and parameters: {'max_features': 1000, 'max_ngram': 5, 'min_df': 0.09573859653883238, 'max_df': 0.8301328704994423}. Best is trial 0 with value: 0.02426948001027788.
[I 2025-04-21 12:21:36,745] Trial 2 finished with value: 0.4013450243812178 and parameters: {'max_features': 49000, 'max_ngram': 1, 'min_df': 0.005988372433102538, 'max_df': 0.8835149547257732}. Best is trial 2 with value: 0.4013450243812178.
[I 2025-04-21 12:28:15,444] Trial 3 finished with value: 0.06266962334566671 and parameters: {'max_features': 49000, 'max_ngram': 5, 'min_df': 0.03616408020

In [39]:
save_optimization_results(
    params=best_params,
    metrics=best_metrics,
    model_type="cbr"
)

## Модель 3: LightFM (Гибридная система)

In [40]:
train_data = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_train.csv")
test_data = pd.read_csv(f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_test.csv")

item_categories = {
    row['product_id']: row['product_group'] 
    for _, row in train_data.iterrows()
}

In [41]:
class FeatureProcessor:
    """
    Класс для параллельной подготовки признаков пользователей и товаров.

    Этот класс обрабатывает сырые данные и создает разреженные матрицы признаков,
    которые могут быть использованы в рекомендательной системе. Поддерживает
    параллельную обработку для ускорения на больших наборах данных.

    Attrs:
        user_col (str): Название колонки с ID пользователей
        item_col (str): Название колонки с ID товаров
        text_cols (List[str]): Список колонок с текстовыми описаниями товаров
        numeric_user_cols (List[str]): Список числовых признаков пользователей
        binary_user_cols (List[str]): Список бинарных признаков пользователей
        n_jobs (int): Количество процессов для параллельной обработки
        user_scaler (StandardScaler): Нормализатор числовых признаков пользователей
        vectorizer (TfidfVectorizer): Векторайзер для текстовых признаков товаров
        user_mapping (Dict[str, int]): Маппинг ID пользователей в индексы
        item_mapping (Dict[str, int]): Маппинг ID товаров в индексы
    """

    def __init__(
        self,
        user_col: str = "buyer_id",
        item_col: str = "product_id",
        text_cols: List[str] = ["product_name", "product_group"],
        numeric_user_cols: List[str] = ["buyer_age"],
        binary_user_cols: List[str] = [
            "buyer_sms_allowed",
            "buyer_emails_allowed",
            "buyer_account_status_loyal",
            "buyer_account_status_unregistered",
            "buyer_account_status_new",
            "buyer_account_status_potential",
            "buyer_account_status_lost",
            "buyer_account_status_sleeping",
            "buyer_is_female",
        ],
        n_jobs: int = -1,
    ):
        """
        Инициализация процессора признаков.

        Args:
            user_col: Название колонки с ID пользователей
            item_col: Название колонки с ID товаров
            text_cols: Список колонок с текстовыми описаниями товаров
            numeric_user_cols: Список числовых признаков пользователей
            binary_user_cols: Список бинарных признаков пользователей
            n_jobs: Количество процессов для параллельной обработки
                   (-1 для использования всех доступных ядер)
        """
        self.user_col = user_col
        self.item_col = item_col
        self.text_cols = text_cols
        self.numeric_user_cols = numeric_user_cols
        self.binary_user_cols = binary_user_cols
        self.n_jobs = multiprocessing.cpu_count() if n_jobs == -1 else n_jobs

        # Инициализация преобразователей признаков
        self.user_scaler = StandardScaler()
        self.vectorizer = TfidfVectorizer(
            max_features=1000,  # Ограничение количества признаков
            ngram_range=(1, 2),  # Униграммы и биграммы
            min_df=2,  # Минимальная частота встречаемости слова
        )

    @staticmethod
    def _combine_text_features(row: pd.Series, text_cols: List[str]) -> str:
        """
        Объединяет текстовые признаки из разных колонок в одну строку.

        Args:
            row: Строка датафрейма с текстовыми признаками
            text_cols: Список колонок с текстовыми данными

        Returns:
            str: Объединенная строка текстовых признаков в нижнем регистре

        Example:
            >>> row = pd.Series({'name': 'Phone', 'group': 'Electronics'})
            >>> _combine_text_features(row, ['name', 'group'])
            'phone electronics'
        """
        return " ".join(str(row[col]).lower() for col in text_cols)

    def _process_user_batch(self, args: Tuple[List[str], pd.DataFrame]) -> np.ndarray:
        """
        Обрабатывает батч пользователей, извлекая их признаки.

        Метод извлекает числовые и бинарные признаки для каждого пользователя
        в батче и объединяет их в единую матрицу.

        Args:
            args: Кортеж, содержащий:
                - список ID пользователей для обработки
                - датафрейм с данными пользователей

        Returns:
            np.ndarray: Матрица признаков размером [len(user_batch) x n_features],
                       где n_features = len(numeric_user_cols) + len(binary_user_cols)
        """
        user_batch, data = args
        features_list = []

        for uid in user_batch:
            user_data = data[data[self.user_col] == uid].iloc[0]

            numeric_features = [user_data[col] for col in self.numeric_user_cols]

            binary_features = [user_data[col] for col in self.binary_user_cols]

            features_list.append(numeric_features + binary_features)

        return np.array(features_list)

    def _prepare_user_features_parallel(self, data: pd.DataFrame) -> sparse.csr_matrix:
        """
        Параллельно подготавливает признаки всех пользователей.

        Метод разбивает пользователей на батчи, параллельно обрабатывает каждый батч,
        нормализует числовые признаки и объединяет результаты в разреженную матрицу.

        Args:
            data: Датафрейм с данными пользователей

        Returns:
            sparse.csr_matrix: Разреженная матрица признаков пользователей размером
                             [n_users x (n_numeric_features + n_binary_features)]
        """
        unique_users = data[self.user_col].unique()

        batch_size = len(unique_users) // self.n_jobs + 1
        user_batches = [
            unique_users[i : i + batch_size]
            for i in range(0, len(unique_users), batch_size)
        ]

        process_args = [(batch, data) for batch in user_batches]

        with ProcessPoolExecutor(max_workers=self.n_jobs) as executor:
            features_batches = list(
                executor.map(self._process_user_batch, process_args)
            )

        all_features = np.vstack(features_batches)

        numeric_features = all_features[:, : len(self.numeric_user_cols)]
        binary_features = all_features[:, len(self.numeric_user_cols) :]

        scaled_numeric = self.user_scaler.fit_transform(numeric_features)

        return sparse.csr_matrix(np.hstack([scaled_numeric, binary_features]))

    def _process_item_batch(
        self, args: Tuple[pd.DataFrame, List[str]]
    ) -> sparse.csr_matrix:
        """
        Обрабатывает батч товаров, создавая их текстовые представления.

        Args:
            args: Кортеж, содержащий:
                - датафрейм с информацией о товарах
                - список колонок с текстовыми описаниями

        Returns:
            sparse.csr_matrix: Разреженная матрица TF-IDF признаков для батча товаров
        """
        item_batch, text_cols = args
        text_features = item_batch.apply(
            self._combine_text_features, args=(text_cols,), axis=1
        )
        return self.vectorizer.transform(text_features)

    def _prepare_item_features_parallel(self, data: pd.DataFrame) -> sparse.csr_matrix:
        """
        Параллельно подготавливает признаки всех товаров.

        Метод сначала обучает TF-IDF векторайзер на всех текстовых данных,
        затем параллельно обрабатывает батчи товаров.

        Args:
            data: Датафрейм с данными о товарах

        Returns:
            sparse.csr_matrix: Разреженная матрица TF-IDF признаков товаров
        """
        unique_items = data[[self.item_col] + self.text_cols].drop_duplicates()

        text_features = unique_items.apply(
            self._combine_text_features, args=(self.text_cols,), axis=1
        )
        self.vectorizer.fit(text_features)

        batch_size = len(unique_items) // self.n_jobs + 1
        item_batches = [
            unique_items.iloc[i : i + batch_size]
            for i in range(0, len(unique_items), batch_size)
        ]

        process_args = [(batch, self.text_cols) for batch in item_batches]

        with ProcessPoolExecutor(max_workers=self.n_jobs) as executor:
            features_batches = list(
                executor.map(self._process_item_batch, process_args)
            )

        return sparse.vstack(features_batches)

    def fit_transform(
        self, data: pd.DataFrame
    ) -> Tuple[sparse.csr_matrix, sparse.csr_matrix]:
        """
        Подготавливает признаки пользователей и товаров из исходных данных.

        Этот метод является основным интерфейсом класса. Он создает маппинги
        идентификаторов в индексы и параллельно подготавливает признаки
        пользователей и товаров.

        Args:
            data: Датафрейм с данными, содержащий информацию о пользователях и товарах

        Returns:
            Tuple[sparse.csr_matrix, sparse.csr_matrix]: Кортеж из двух матриц:
                - матрица признаков пользователей
                - матрица признаков товаров

        Example:
            >>> processor = FeatureProcessor()
            >>> user_features, item_features = processor.fit_transform(data)
            >>> print(f"User features shape: {user_features.shape}")
            >>> print(f"Item features shape: {item_features.shape}")
        """
        logging.info("Создание маппингов идентификаторов в индексы...")
        self.user_mapping = {
            uid: idx for idx, uid in enumerate(data[self.user_col].unique())
        }
        self.item_mapping = {
            iid: idx for idx, iid in enumerate(data[self.item_col].unique())
        }

        n_users = len(self.user_mapping)
        n_items = len(self.item_mapping)

        user_features = self._prepare_user_features_parallel(data)

        item_features = self._prepare_item_features_parallel(data)

        if user_features.shape[0] != n_users:
            raise ValueError(
                f"Ошибка в подготовке признаков пользователей: "
                f"получено {user_features.shape[0]} строк вместо {n_users}"
            )

        if item_features.shape[0] != n_items:
            raise ValueError(
                f"Ошибка в подготовке признаков товаров: "
                f"получено {item_features.shape[0]} строк вместо {n_items}"
            )

        return user_features, item_features

    def save_features(
        self,
        path: str,
        user_features: sparse.csr_matrix,
        item_features: sparse.csr_matrix,
    ) -> None:
        """
        Сохраняет все подготовленные признаки, маппинги и преобразователи в файл.

        Args:
            path (str): Путь для сохранения файла
            user_features (sparse.csr_matrix): Матрица признаков пользователей
            item_features (sparse.csr_matrix): Матрица признаков товаров

        Note:
            Сохраняет следующие данные:
            - Матрицы признаков пользователей и товаров
            - Маппинги ID пользователей и товаров в индексы
            - Нормализатор числовых признаков пользователей
            - TF-IDF векторайзер для текстовых признаков товаров
            - Конфигурацию процессора (названия колонок и другие параметры)

        Example:
            >>> processor = FeatureProcessor()
            >>> user_features, item_features = processor.fit_transform(data)
            >>> processor.save_features("features.npz", user_features, item_features)
        """
        if not hasattr(self, "user_mapping") or not hasattr(self, "item_mapping"):
            raise ValueError(
                "Сначала необходимо подготовить признаки с помощью fit_transform"
            )

        path = os.path.expanduser(path)

        os.makedirs(os.path.dirname(path), exist_ok=True)

        np.savez_compressed(
            path,
            # Матрицы признаков
            user_features_data=user_features.data,
            user_features_indices=user_features.indices,
            user_features_indptr=user_features.indptr,
            user_features_shape=user_features.shape,
            item_features_data=item_features.data,
            item_features_indices=item_features.indices,
            item_features_indptr=item_features.indptr,
            item_features_shape=item_features.shape,
            # Маппинги
            user_mapping=self.user_mapping,
            item_mapping=self.item_mapping,
            # Преобразователи
            user_scaler=self.user_scaler,
            vectorizer=self.vectorizer,
            # Конфигурация
            config=dict(
                user_col=self.user_col,
                item_col=self.item_col,
                text_cols=self.text_cols,
                numeric_user_cols=self.numeric_user_cols,
                binary_user_cols=self.binary_user_cols,
                n_jobs=self.n_jobs,
            ),
        )

        logging.info(f"Признаки успешно сохранены в {path}")

    @classmethod
    def load_features(
        cls, path: str
    ) -> Tuple["FeatureProcessor", sparse.csr_matrix, sparse.csr_matrix]:
        """
        Загружает сохраненные признаки, маппинги и преобразователи из файла.

        Args:
            path (str): Путь к файлу с сохраненными признаками

        Returns:
            Tuple[FeatureProcessor, sparse.csr_matrix, sparse.csr_matrix]:
                - Инициализированный процессор признаков
                - Матрица признаков пользователей
                - Матрица признаков товаров

        Example:
            >>> processor, user_features, item_features = FeatureProcessor.load_features("features.npz")
            >>> print(f"Загружено {user_features.shape[0]} пользователей и {item_features.shape[0]} товаров")

        Raises:
            FileNotFoundError: Если файл не найден
            ValueError: Если файл содержит некорректные данные
        """
        path = os.path.expanduser(path)

        try:
            loaded = np.load(path, allow_pickle=True)

            config = loaded["config"].item()

            processor = cls(
                user_col=config["user_col"],
                item_col=config["item_col"],
                text_cols=config["text_cols"],
                numeric_user_cols=config["numeric_user_cols"],
                binary_user_cols=config["binary_user_cols"],
                n_jobs=config["n_jobs"],
            )

            processor.user_mapping = loaded["user_mapping"].item()
            processor.item_mapping = loaded["item_mapping"].item()

            processor.user_scaler = loaded["user_scaler"].item()
            processor.vectorizer = loaded["vectorizer"].item()

            user_features = sparse.csr_matrix(
                (
                    loaded["user_features_data"],
                    loaded["user_features_indices"],
                    loaded["user_features_indptr"],
                ),
                shape=tuple(loaded["user_features_shape"]),
            )

            item_features = sparse.csr_matrix(
                (
                    loaded["item_features_data"],
                    loaded["item_features_indices"],
                    loaded["item_features_indptr"],
                ),
                shape=tuple(loaded["item_features_shape"]),
            )

            logging.info(f"Признаки успешно загружены из {path}")
            logging.info(f"Размерность признаков пользователей: {user_features.shape}")
            logging.info(f"Размерность признаков товаров: {item_features.shape}")

            return processor, user_features, item_features

        except FileNotFoundError:
            raise FileNotFoundError(f"Файл {path} не найден")
        except Exception as e:
            raise ValueError(f"Ошибка при загрузке признаков: {str(e)}")

In [42]:
feature_processor = FeatureProcessor(n_jobs=-1)
user_features, item_features = feature_processor.fit_transform(train_data)

feature_processor.save_features(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_features.npz",
    user_features,
    item_features
)

2025-04-21 14:14:27 - INFO - Создание маппингов идентификаторов в индексы...
2025-04-21 14:26:38 - INFO - Признаки успешно сохранены в /home/jupyter/datasphere/s3/pb-ml-gamma/stat/2821f47e-961c-25b9-a94c-f789a14f7a57_2025-03-12_features.npz


In [43]:
feature_processor, loaded_user_features, loaded_item_features = FeatureProcessor.load_features(
   f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_features.npz"
)

2025-04-21 14:26:38 - INFO - Признаки успешно загружены из /home/jupyter/datasphere/s3/pb-ml-gamma/stat/2821f47e-961c-25b9-a94c-f789a14f7a57_2025-03-12_features.npz
2025-04-21 14:26:38 - INFO - Размерность признаков пользователей: (114454, 10)
2025-04-21 14:26:38 - INFO - Размерность признаков товаров: (15053, 1000)


In [44]:
class HybridRecommender:
    """
    Гибридная рекомендательная система на основе LightFM
    """

    def __init__(self, random_state: int = 42):
        self.random_state = random_state
        self.model = None
        self.user_mapping = None
        self.item_mapping = None
        self.user_features_matrix = None
        self.item_features_matrix = None

    def fit(
        self,
        train_data: pd.DataFrame,
        user_features: sparse.csr_matrix,
        item_features: sparse.csr_matrix,
        user_mapping: Dict[str, int],
        item_mapping: Dict[str, int],
        model_params: Dict,
        weight_col: str = "product_count",
    ) -> None:
        """
        Обучение модели

        Args:
            train_data: Датафрейм с историей взаимодействий
            user_features: Разреженная матрица признаков пользователей
            item_features: Разреженная матрица признаков товаров
            user_mapping: Маппинг ID пользователей в индексы
            item_mapping: Маппинг ID товаров в индексы
            model_params: Параметры модели
            weight_col: Название колонки с весами взаимодействий
        """
        self.user_mapping = user_mapping
        self.item_mapping = item_mapping

        n_users = len(user_mapping)
        n_items = len(item_mapping)

        if user_features.shape[0] != n_users:
            raise ValueError(
                f"Количество строк в матрице признаков пользователей ({user_features.shape[0]}) "
                f"не соответствует количеству пользователей ({n_users})"
            )

        if item_features.shape[0] != n_items:
            raise ValueError(
                f"Количество строк в матрице признаков товаров ({item_features.shape[0]}) "
                f"не соответствует количеству товаров ({n_items})"
            )

        self.user_features_matrix = user_features
        self.item_features_matrix = item_features

        interactions = sparse.coo_matrix(
            (
                train_data[weight_col].values,
                (
                    train_data["buyer_id"].map(user_mapping).values,
                    train_data["product_id"].map(item_mapping).values,
                ),
            ),
            shape=(n_users, n_items),
        ).tocsr()

        self.model = LightFM(
            loss=model_params.get("loss", "warp"),
            no_components=model_params["no_components"],
            learning_rate=model_params["learning_rate"],
            user_alpha=model_params["user_alpha"],
            item_alpha=model_params["item_alpha"],
            random_state=self.random_state,
        )

        self.model.fit(
            interactions=interactions,
            user_features=self.user_features_matrix,
            item_features=self.item_features_matrix,
            epochs=model_params["epochs"],
            verbose=True,
        )

    def recommend(self, users: List[str], k: int) -> Dict[str, List[str]]:
        """
        Генерирует персонализированные рекомендации для списка пользователей.

        Args:
            users (List[str]): Список идентификаторов пользователей
            k (int): Количество рекомендаций для каждого пользователя

        Returns:
            Dict[str, List[str]]: Словарь рекомендаций для каждого пользователя
        """
        if self.model is None:
            raise ValueError("Модель не обучена. Сначала выполните fit()")

        recommendations = {}
        reverse_item_mapping = {v: k for k, v in self.item_mapping.items()}
        n_items = len(self.item_mapping)
        item_ids = np.arange(n_items)
        
        for _, user_id in enumerate(users):
            if user_id in self.user_mapping:
                user_idx = self.user_mapping[user_id]
                user_ids = np.full_like(item_ids, user_idx)
                
                scores = self.model.predict(
                    user_ids=user_ids,
                    item_ids=item_ids,
                    user_features=self.user_features_matrix,
                    item_features=self.item_features_matrix
                )
                
                top_items_indices = np.argsort(-scores)[:k]
                recommendations[user_id] = [
                    reverse_item_mapping[idx] 
                    for idx in top_items_indices
                ]
            else:
                recommendations[user_id] = []
        
        return recommendations

    def save_model(self, path: str) -> None:
        """
        Сохраняет обученную модель и все необходимые данные в файл.

        Метод сохраняет полное состояние рекомендательной системы, включая:
        - Обученную модель LightFM
        - Маппинги идентификаторов пользователей и товаров
        - Матрицы признаков пользователей и товаров
        - Параметр random_state для воспроизводимости

        Args:
            path (str):
                Путь для сохранения модели. Может быть как абсолютным,
                так и относительным. Рекомендуется использовать расширение
                '.joblib' для файла.

        Note:
            1. Метод использует библиотеку joblib для сериализации, которая
            эффективно работает с научными объектами Python (numpy arrays,
            scipy sparse matrices).

            2. Сохраняются следующие компоненты:
            - model: обученная модель LightFM
            - user_mapping: словарь {user_id -> индекс}
            - item_mapping: словарь {item_id -> индекс}
            - user_features_matrix: разреженная матрица признаков пользователей
            - item_features_matrix: разреженная матрица признаков товаров
            - random_state: значение для воспроизводимости

            3. Все компоненты необходимы для корректной работы метода recommend()
            после загрузки модели.

        Example:
            >>> recommender = HybridRecommender()
            >>> recommender.fit(train_data, user_features, item_features, ...)
            >>> recommender.save_model("hybrid_model_v1.joblib")
        """
        model_data = {
            "model": self.model,
            "user_mapping": self.user_mapping,
            "item_mapping": self.item_mapping,
            "user_features_matrix": self.user_features_matrix,
            "item_features_matrix": self.item_features_matrix,
            "random_state": self.random_state,
        }
        joblib.dump(model_data, path)

    @classmethod
    def load_model(cls, path: str) -> "HybridRecommender":
        """
        Загружает сохраненную модель и все необходимые данные из файла.

        Этот метод является классовым методом, который создает новый экземпляр
        HybridRecommender и инициализирует его сохраненными данными. Загружает
        все компоненты, необходимые для генерации рекомендаций.

        Args:
            path (str):
                Путь к файлу с сохраненной моделью. Может быть как абсолютным,
                так и относительным путем. Файл должен быть создан методом save_model.

        Returns:
            HybridRecommender:
                Полностью инициализированный экземпляр рекомендательной системы,
                готовый к генерации рекомендаций.

        Note:
            1. Загружаются следующие компоненты:
            - model: обученная модель LightFM
            - user_mapping: словарь {user_id -> индекс}
            - item_mapping: словарь {item_id -> индекс}
            - user_features_matrix: разреженная матрица признаков пользователей
            - item_features_matrix: разреженная матрица признаков товаров
            - random_state: значение для воспроизводимости

            2. После загрузки модель готова к использованию без необходимости
            повторного обучения.

            3. Все компоненты проверяются на корректность при загрузке.

        Example:
            >>> # Загрузка ранее сохраненной модели
            >>> recommender = HybridRecommender.load_model("hybrid_model_v1.joblib")
            >>> # Генерация рекомендаций
            >>> recommendations = recommender.recommend(users=['user1', 'user2'], k=10)

        Raises:
            FileNotFoundError: Если файл с моделью не найден
            ValueError: Если файл содержит некорректные или неполные данные
            Exception: При других ошибках загрузки

        See Also:
            save_model: Метод для сохранения модели
        """
        try:
            model_data = joblib.load(path)

            required_components = {
                "model",
                "user_mapping",
                "item_mapping",
                "user_features_matrix",
                "item_features_matrix",
                "random_state",
            }

            if not all(comp in model_data for comp in required_components):
                missing = required_components - set(model_data.keys())
                raise ValueError(
                    f"Файл с моделью не содержит необходимых компонентов: {missing}"
                )

            recommender = cls(random_state=model_data["random_state"])

            # Инициализируем все компоненты
            recommender.model = model_data["model"]
            recommender.user_mapping = model_data["user_mapping"]
            recommender.item_mapping = model_data["item_mapping"]
            recommender.user_features_matrix = model_data["user_features_matrix"]
            recommender.item_features_matrix = model_data["item_features_matrix"]

            logging.info(f"Модель успешно загружена из {path}")
            logging.info(f"Количество пользователей: {len(recommender.user_mapping)}")
            logging.info(f"Количество товаров: {len(recommender.item_mapping)}")

            return recommender

        except FileNotFoundError:
            raise FileNotFoundError(f"Файл с моделью не найден: {path}")
        except Exception as e:
            raise Exception(f"Ошибка при загрузке модели: {str(e)}")

In [45]:
def optimize_hybrid_parameters(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    features_path: str,
    metrics_calculator: MetricsCalculator,
    n_trials: int = 50,
    timeout: int = 3600,
    random_state: int = 42,
) -> Tuple[Dict, Dict[str, float], HybridRecommender]:
    """
    Оптимизация параметров гибридного рекомендера с помощью Optuna.

    Args:
        train_data: Данные для обучения
        test_data: Данные для тестирования
        features_path: Путь к сохраненным признакам
        metrics_calculator: Калькулятор метрик
        n_trials: Количество итераций оптимизации
        timeout: Максимальное время оптимизации в секундах
        random_state: Фиксация случайности

    Returns:
        Tuple[Dict, Dict[str, float], HybridRecommender]:
            - Лучшие параметры
            - Финальные метрики
            - Обученная модель с лучшими параметрами
    """
    logging.info("Загрузка предварительно подготовленных признаков...")
    feature_processor, user_features, item_features = FeatureProcessor.load_features(
        features_path
    )

    def objective(trial: optuna.Trial) -> float:
        params = {
            "no_components": trial.suggest_int("no_components", 32, 256, step=32),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
            "user_alpha": trial.suggest_float("user_alpha", 1e-6, 1e-2, log=True),
            "item_alpha": trial.suggest_float("item_alpha", 1e-6, 1e-2, log=True),
            "epochs": trial.suggest_int("epochs", 10, 50, step=20),
            "loss": "warp",
        }

        logging.info(f"Trial {trial.number}: Начало обучения с параметрами {params}")

        recommender = HybridRecommender(random_state=random_state)
        recommender.fit(
            train_data=train_data,
            user_features=user_features,
            item_features=item_features,
            user_mapping=feature_processor.user_mapping,
            item_mapping=feature_processor.item_mapping,
            model_params=params,
        )

        test_users = test_data["buyer_id"].unique()
        logging.info(
            f"Trial {trial.number}: Генерация рекомендаций для {len(test_users)} пользователей"
        )

        recommendations = recommender.recommend(
            users=test_users, k=max(metrics_calculator.k_values)
        )

        metrics = metrics_calculator.calculate(
            recommendations=recommendations,
            train_data=train_data,
            test_data=test_data,
            item_categories=item_categories,
        )

        target_k = max(metrics_calculator.k_values)
        recall = metrics[f"recall_{target_k}"]

        logging.info(f"Trial {trial.number}: Recall@{target_k} = {recall:.4f}")
        return recall

    study = optuna.create_study(
        direction="maximize", sampler=optuna.samplers.TPESampler(seed=random_state)
    )

    logging.info("Начало процесса оптимизации...")
    study.optimize(
        objective,
        n_trials=n_trials,
        timeout=timeout,
        show_progress_bar=True,
        n_jobs=-1,
    )

    completed_trials = [
        t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE
    ]
    if not completed_trials:
        raise ValueError(
            "Ни один trial не был успешно завершен. Проверьте логи для деталей."
        )

    best_params = study.best_params
    best_params["loss"] = "warp"

    logging.info(f"Лучшие параметры: {best_params}")
    logging.info(f"Лучшее значение целевой метрики: {study.best_value:.4f}")

    logging.info("Обучение финальной модели с лучшими параметрами...")
    best_recommender = HybridRecommender(random_state=random_state)
    best_recommender.fit(
        train_data=train_data,
        user_features=user_features,
        item_features=item_features,
        user_mapping=feature_processor.user_mapping,
        item_mapping=feature_processor.item_mapping,
        model_params=best_params,
    )

    test_users = test_data["buyer_id"].unique()
    final_recommendations = best_recommender.recommend(
        users=test_users, k=max(metrics_calculator.k_values)
    )

    final_metrics = metrics_calculator.calculate(
        recommendations=final_recommendations,
        train_data=train_data,
        test_data=test_data,
        item_categories=item_categories,
    )

    logging.info("Финальные метрики:")
    for metric_name, value in final_metrics.items():
        logging.info(f"{metric_name}: {value:.4f}")

    return best_params, final_metrics, best_recommender

In [46]:
metrics_calculator = MetricsCalculator([10, 100, 1000])

features_path = (
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_features.npz"
)

metrics_calculator = MetricsCalculator([10, 100, 1000])

best_params, best_metrics, best_model = optimize_hybrid_parameters(
    train_data=train_data,
    test_data=test_data,
    features_path=features_path,
    metrics_calculator=metrics_calculator,
    n_trials=50,
    timeout=7200,
    random_state=42
)

logging.info(f"Лучшие параметры: {best_params}")
logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"Hit Rate@{k}: {best_metrics[f'hit_rate_{k}']:.4f}")
    logging.info(f"Precision@{k}: {best_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {best_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {best_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {best_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {best_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")


save_path = os.path.expanduser(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_hybrid_recommender.joblib"
)
best_model.save_model(save_path)

2025-04-21 14:26:38 - INFO - Загрузка предварительно подготовленных признаков...
2025-04-21 14:26:39 - INFO - Признаки успешно загружены из /home/jupyter/datasphere/s3/pb-ml-gamma/stat/2821f47e-961c-25b9-a94c-f789a14f7a57_2025-03-12_features.npz
2025-04-21 14:26:39 - INFO - Размерность признаков пользователей: (114454, 10)
2025-04-21 14:26:39 - INFO - Размерность признаков товаров: (15053, 1000)
[I 2025-04-21 14:26:39,026] A new study created in memory with name: no-name-a9ce668c-cfa2-4fd7-923a-8c8e91bc8bbb
2025-04-21 14:26:39 - INFO - Начало процесса оптимизации...
  0%|          | 0/50 [00:00<?, ?it/s]2025-04-21 14:26:39 - INFO - Trial 0: Начало обучения с параметрами {'no_components': 128, 'learning_rate': 0.05596134372088848, 'user_alpha': 0.00036979433025404807, 'item_alpha': 2.59436427204147e-05, 'epochs': 30, 'loss': 'warp'}
2025-04-21 14:26:39 - INFO - Trial 1: Начало обучения с параметрами {'no_components': 96, 'learning_rate': 0.003919302888960402, 'user_alpha': 0.00014976641

[I 2025-04-21 14:50:22,810] Trial 12 finished with value: 0.6550431741385778 and parameters: {'no_components': 32, 'learning_rate': 0.01230297356185196, 'user_alpha': 7.221072287573657e-05, 'item_alpha': 0.0033240102455081573, 'epochs': 30}. Best is trial 12 with value: 0.6550431741385778.










Epoch: 100%|██████████| 50/50 [23:44<00:00, 28.49s/it]
2025-04-21 14:50:33 - INFO - Trial 8: Генерация рекомендаций для 60050 пользователей
2025-04-21 14:55:19 - INFO - Trial 4: Recall@1000 = 0.6573

Best trial: 4. Best value: 0.657268:   4%|▍         | 2/50 [28:40<10:08:44, 760.93s/it, 1423.86/7200 seconds]  

[I 2025-04-21 14:55:19,709] Trial 4 finished with value: 0.6572679176087614 and parameters: {'no_components': 32, 'learning_rate': 0.0020733687176298756, 'user_alpha': 0.0021734737185078604, 'item_alpha': 0.0002019602516876208, 'epochs': 50}. Best is trial 4 with value: 0.6572679176087614.


Best trial: 4. Best value: 0.657268:   4%|▍         | 2/50 [28:40<10:08:44, 760.93s/it, 1720.74/7200 seconds]2025-04-21 14:55:19 - INFO - Trial 33: Начало обучения с параметрами {'no_components': 192, 'learning_rate': 0.0011069933077867517, 'user_alpha': 0.0019978890677673864, 'item_alpha': 1.9070736577318194e-05, 'epochs': 10, 'loss': 'warp'}




2025-04-21 14:56:16 - INFO - Trial 18: Recall@1000 = 0.6548

                                                                                                             

Best trial: 4. Best value: 0.657268:   6%|▌         | 3/50 [29:38<5:44:27, 439.72s/it, 1778.21/7200 seconds] 2025-04-21 14:56:17 - INFO - Trial 34: Начало обучения с параметрами {'no_components': 128, 'learning_rate': 0.005476651186840881, 'user_alpha': 2.6000845804740728e-05, 'item_alpha': 6.571666645736875e-05, 'epochs': 30, 'loss': 'warp'}


...

2025-04-21 16:25:40 - INFO - Trial 45: Recall@1000 = 0.6562
Best trial: 46. Best value: 0.658118:  98%|█████████▊| 49/50 [1:59:01<01:22, 82.64s/it, 7141.67/7200 seconds]

[I 2025-04-21 16:25:40,690] Trial 45 finished with value: 0.656210224175812 and parameters: {'no_components': 224, 'learning_rate': 0.002188831166101703, 'user_alpha': 1.8944201193059469e-06, 'item_alpha': 1.0572995483200955e-06, 'epochs': 50}. Best is trial 46 with value: 0.6581181051351433.


2025-04-21 16:28:29 - INFO - Trial 49: Recall@1000 = 0.6556
Best trial: 46. Best value: 0.658118: 100%|██████████| 50/50 [2:01:51<00:00, 146.22s/it, 7311.05/7200 seconds]
2025-04-21 16:28:30 - INFO - Лучшие параметры: {'no_components': 224, 'learning_rate': 0.002380830436675144, 'user_alpha': 1.3638010461914722e-06, 'item_alpha': 5.12376977673685e-05, 'epochs': 10, 'loss': 'warp'}
2025-04-21 16:28:30 - INFO - Лучшее значение целевой метрики: 0.6581
2025-04-21 16:28:30 - INFO - Обучение финальной модели с лучшими параметрами...


[I 2025-04-21 16:28:30,072] Trial 49 finished with value: 0.6555594758515447 and parameters: {'no_components': 256, 'learning_rate': 0.0019092892222045952, 'user_alpha': 1.4153316853168952e-06, 'item_alpha': 1.0966773496731228e-06, 'epochs': 10}. Best is trial 46 with value: 0.6581181051351433.


Epoch: 100%|██████████| 10/10 [03:00<00:00, 18.08s/it]
2025-04-21 16:55:11 - INFO - Финальные метрики:
2025-04-21 16:55:11 - INFO - hit_rate_10: 0.0631
2025-04-21 16:55:11 - INFO - precision_10: 0.0065
2025-04-21 16:55:11 - INFO - recall_10: 0.0168
2025-04-21 16:55:11 - INFO - diversity_10: 0.0018
2025-04-21 16:55:11 - INFO - novelty_10: 0.5746
2025-04-21 16:55:11 - INFO - serendipity_10: 0.0023
2025-04-21 16:55:11 - INFO - hit_rate_100: 0.5800
2025-04-21 16:55:11 - INFO - precision_100: 0.0099
2025-04-21 16:55:11 - INFO - recall_100: 0.2349
2025-04-21 16:55:11 - INFO - diversity_100: 0.0222
2025-04-21 16:55:11 - INFO - novelty_100: 0.5748
2025-04-21 16:55:11 - INFO - serendipity_100: 0.0013
2025-04-21 16:55:11 - INFO - hit_rate_1000: 0.9067
2025-04-21 16:55:11 - INFO - precision_1000: 0.0030
2025-04-21 16:55:11 - INFO - recall_1000: 0.6581
2025-04-21 16:55:11 - INFO - diversity_1000: 0.1377
2025-04-21 16:55:11 - INFO - novelty_1000: 0.6647
2025-04-21 16:55:11 - INFO - serendipity_1000

In [47]:
save_optimization_results(
    params=best_params,
    metrics=best_metrics,
    model_type="hybrid"
)

## Сохранение лучших моделей

In [48]:
logging.info("Обучение модели ALS...")
als = ALSRecommender(weight_col="product_count")
als.fit(
    train_data,
    {
        "factors": 32,
        "regularization": 0.8442524315317257,
        "alpha": 0.10453076958875206,
        "iterations": 14,
    },
)

logging.info("Сохранение модели ALS...")
als.save_model(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_als_recommender.joblib"
)

logging.info("Загрузка модели ALS...")
als = ALSRecommender.load_model(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_als_recommender.joblib"
)

logging.info("Генерация финальных рекомендаций...")
test_users = test_data["buyer_id"].unique()
final_recommendations = als.recommend(
    users=test_users, k=max(metrics_calculator.k_values)
)

logging.info("Сохранение финальных рекомендаций...")
als_recommendations = (
    pd.DataFrame(
        {
            "buyer_id": final_recommendations.keys(),
            "product_id": final_recommendations.values(),
        }
    )
    .explode("product_id")
    .reset_index(drop=True)
)
als_recommendations.to_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_als_recommendations.csv",
    index=False,
)

logging.info("Расчет финальных метрик...")
final_metrics = metrics_calculator.calculate(
    recommendations=final_recommendations,
    train_data=train_data,
    test_data=test_data,
    item_categories=item_categories,
)

logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"Hit Rate@{k}: {final_metrics[f'hit_rate_{k}']:.4f}")
    logging.info(f"Precision@{k}: {final_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {final_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {final_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {final_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {final_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")

2025-04-21 16:55:12 - INFO - Обучение модели ALS...
100%|██████████| 14/14 [00:18<00:00,  1.30s/it]
2025-04-21 16:55:32 - INFO - Сохранение модели ALS...
2025-04-21 16:55:35 - INFO - Загрузка модели ALS...
2025-04-21 16:55:35 - INFO - Генерация финальных рекомендаций...
2025-04-21 16:58:37 - INFO - Сохранение финальных рекомендаций...
2025-04-21 17:03:06 - INFO - Расчет финальных метрик...
2025-04-21 17:03:32 - INFO - Результаты:
2025-04-21 17:03:32 - INFO - Метрики для K=10:
2025-04-21 17:03:32 - INFO - Hit Rate@10: 0.4501
2025-04-21 17:03:32 - INFO - Precision@10: 0.0694
2025-04-21 17:03:32 - INFO - Recall@10: 0.1638
2025-04-21 17:03:32 - INFO - Diversity@10: 0.0051
2025-04-21 17:03:32 - INFO - Novelty@10: 0.3040
2025-04-21 17:03:32 - INFO - Serendipity@10: 0.0372
2025-04-21 17:03:32 - INFO - --------------------------------
2025-04-21 17:03:32 - INFO - Метрики для K=100:
2025-04-21 17:03:32 - INFO - Hit Rate@100: 0.7312
2025-04-21 17:03:32 - INFO - Precision@100: 0.0169
2025-04-21 1

In [49]:
logging.info("Обучение модели CBR...")
cbr = ContentBasedRecommender(weight_col="product_count")
cbr.fit(
    train_data,
    {
        "max_features": 25000,
        "max_ngram": 3,
        "min_df": 0.0013045157897118098,
        "max_df": 0.6771829780010523,
    },
)

logging.info("Сохранение модели CBR...")
cbr.save_model(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_cbr_recommender.joblib"
)

logging.info("Загрузка модели CBR...")
cbr = ContentBasedRecommender.load_model(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_cbr_recommender.joblib"
)

logging.info("Генерация финальных рекомендаций...")
test_users = test_data["buyer_id"].unique()
final_recommendations = cbr.recommend(
    users=test_users, k=max(metrics_calculator.k_values)
)

logging.info("Сохранение финальных рекомендаций...")
cbr_recommendations = (
    pd.DataFrame(
        {
            "buyer_id": final_recommendations.keys(),
            "product_id": final_recommendations.values(),
        }
    )
    .explode("product_id")
    .reset_index(drop=True)
)
cbr_recommendations.to_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_cbr_recommendations.csv",
    index=False,
)

logging.info("Расчет финальных метрик...")
final_metrics = metrics_calculator.calculate(
    recommendations=final_recommendations,
    train_data=train_data,
    test_data=test_data,
    item_categories=item_categories,
)

logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"Hit Rate@{k}: {final_metrics[f'hit_rate_{k}']:.4f}")
    logging.info(f"Precision@{k}: {final_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {final_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {final_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {final_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {final_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")

2025-04-21 17:03:32 - INFO - Обучение модели CBR...
2025-04-21 17:08:45 - INFO - Сохранение модели CBR...
2025-04-21 17:09:12 - INFO - Загрузка модели CBR...
2025-04-21 17:09:31 - INFO - Генерация финальных рекомендаций...
2025-04-21 17:11:36 - INFO - Сохранение финальных рекомендаций...
2025-04-21 17:16:07 - INFO - Расчет финальных метрик...
2025-04-21 17:16:42 - INFO - Результаты:
2025-04-21 17:16:42 - INFO - Метрики для K=10:
2025-04-21 17:16:42 - INFO - Hit Rate@10: 0.4628
2025-04-21 17:16:42 - INFO - Precision@10: 0.0560
2025-04-21 17:16:42 - INFO - Recall@10: 0.1686
2025-04-21 17:16:42 - INFO - Diversity@10: 0.0020
2025-04-21 17:16:42 - INFO - Novelty@10: 0.6508
2025-04-21 17:16:42 - INFO - Serendipity@10: 0.0433
2025-04-21 17:16:42 - INFO - --------------------------------
2025-04-21 17:16:42 - INFO - Метрики для K=100:
2025-04-21 17:16:42 - INFO - Hit Rate@100: 0.6865
2025-04-21 17:16:42 - INFO - Precision@100: 0.0120
2025-04-21 17:16:42 - INFO - Recall@100: 0.3145
2025-04-21 1

In [50]:
feature_processor, user_features, item_features = FeatureProcessor.load_features(
   f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_features.npz"
)

logging.info("Обучение модели LightFM...")
lfm = HybridRecommender()
lfm.fit(
    train_data=train_data,
    user_features=user_features,
    item_features=item_features,
    user_mapping=feature_processor.user_mapping,
    item_mapping=feature_processor.item_mapping,
    model_params={
        "no_components": 192,
        "learning_rate": 0.006304562072713876,
        "user_alpha": 0.003662155699207385,
        "item_alpha": 7.015131383224363e-05,
        "epochs": 10,
        "loss": "warp",
    },
)

logging.info("Сохранение модели LightFM...")
lfm.save_model(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_lfm_recommender.joblib"
)

logging.info("Загрузка модели LightFM...")
lfm = HybridRecommender.load_model(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_lfm_recommender.joblib"
)

logging.info("Генерация финальных рекомендаций...")
test_users = test_data["buyer_id"].unique()
final_recommendations = lfm.recommend(
    users=test_users, k=max(metrics_calculator.k_values)
)

logging.info("Сохранение финальных рекомендаций...")
lfm_recommendations = (
    pd.DataFrame(
        {
            "buyer_id": final_recommendations.keys(),
            "product_id": final_recommendations.values(),
        }
    )
    .explode("product_id")
    .reset_index(drop=True)
)
lfm_recommendations.to_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_lfm_recommendations.csv",
    index=False,
)

logging.info("Расчет финальных метрик...")
final_metrics = metrics_calculator.calculate(
    recommendations=final_recommendations,
    train_data=train_data,
    test_data=test_data,
    item_categories=item_categories,
)

logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"Hit Rate@{k}: {final_metrics[f'hit_rate_{k}']:.4f}")
    logging.info(f"Precision@{k}: {final_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {final_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {final_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {final_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {final_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")

2025-04-21 17:16:43 - INFO - Признаки успешно загружены из /home/jupyter/datasphere/s3/pb-ml-gamma/stat/2821f47e-961c-25b9-a94c-f789a14f7a57_2025-03-12_features.npz
2025-04-21 17:16:43 - INFO - Размерность признаков пользователей: (114454, 10)
2025-04-21 17:16:43 - INFO - Размерность признаков товаров: (15053, 1000)
2025-04-21 17:16:43 - INFO - Обучение модели LightFM...
Epoch: 100%|██████████| 10/10 [02:27<00:00, 14.80s/it]
2025-04-21 17:19:11 - INFO - Сохранение модели LightFM...
2025-04-21 17:19:12 - INFO - Загрузка модели LightFM...
2025-04-21 17:19:12 - INFO - Модель успешно загружена из ~/datasphere/s3/pb-ml-gamma/stat/2821f47e-961c-25b9-a94c-f789a14f7a57_2025-03-12_lfm_recommender.joblib
2025-04-21 17:19:12 - INFO - Количество пользователей: 114454
2025-04-21 17:19:12 - INFO - Количество товаров: 15053
2025-04-21 17:19:12 - INFO - Генерация финальных рекомендаций...
2025-04-21 17:38:03 - INFO - Сохранение финальных рекомендаций...
2025-04-21 17:42:29 - INFO - Расчет финальных ме

## Сравнение моделей

### Лучшие гиперпараметры моделей

**Alternating Least Squares (Коллаборативная фильтрация)**:
```json
{
    'factors': 32, 
    'regularization': 0.8442524315317257, 
    'alpha': 0.10453076958875206, 
    'iterations': 14
}
```

**Content-Based (Контентная фильтрация)**:
```json
{
    'max_features': 25000, 
    'max_ngram': 3, 
    'min_df': 0.0013045157897118098, 
    'max_df': 0.6771829780010523
}
```

**LightFM (Гибридная система)**:
```json
{
    'no_components': 192, 
    'learning_rate': 0.006304562072713876, 
    'user_alpha': 0.003662155699207385, 
    'item_alpha': 7.015131383224363e-05, 
    'epochs': 10, 
    'loss': 'warp'
}
```


### Метрики работы моделей

| Метрика  | ALS  |  CB | Hybrid  |
|---|---|---|---|
|  Время обучения |  1 мин. |  7 мин. |  3 мин. |
|  Время получения предсказаний |  4 мин. |  4 мин. |  22 мин. |
|  Recall@10 |  0.1638 |  0.1686 |  0.0178 |
|  Recall@100 |  0.3759 |  0.3145 |  0.2268 |
|  Recall@1000 |  0.7612 |  0.5667 |  0.6564 |
|  Hit Rate@10 |  0.4501 |  0.4628 |  0.0670 |
|  Hit Rate@100 |  0.7313 |  0.6865 |  0.5673 |
|  Hit Rate@1000 |  0.9484 |  0.8672 |  0.9077 |
|  Precision@10 |  0.0694 |  0.0560 |  0.0069 |
|  Precision@100 |  0.0169 |  0.0120 |  0.0096 |
|  Precision@1000 |  0.0037 |  0.0026 |  0.0030 |
|  Diversity@10 |  0.0051 |  0.0020 |  0.0018 |
|  Diversity@100 |  0.0358 |  0.0137 |  0.0213 |
|  Diversity@1000 |  0.1882 | 0.1423 |  0.1344 |
|  Novelty@10 |  0.3039 |  0.6508 |  0.5722 |
|  Novelty@100 |  0.3593 |  0.7173 |  0.5728 |
|  Novelty@1000 |  0.4997 |  0.7616 |  0.6646 |
|  Serendipity@10 |  0.0372 |  0.0433 |  0.0023 |
|  Serendipity@100 |  0.0046 |  0.0059 |  0.0013 |
|  Serendipity@1000 |  0.0003 |  0.0004 |  0.0001 |

**Сравнение временных затрат производилось на одинаковых датасетах и одинаковом оборудовании на платформе Intel Ice Lake с 4 vCPU и 32 Гб RAM**

**В метриках не указано, что для модели LightFM требуется предварительно подготовить признаки, что занимает около 90 минут.**

## Итоги

В ходе работы было произведено:
- анализ подходящих моделей и алгоритмов машинного обучения для генерации кандидатов
- реализована модель коллаборативной фильтрации - Alternating Least Squares
- реализована модель контентной фильтрации
- реализована гибридная модель - LastFM и подготовка признаков для неё
- проведено обучение моделей
- выполнен анализ результатов работы моделей

Согласно результатам работы моделей, Alternating Least Squares является наиболее подходящим решением с учётом бизнес-требований и имеющихся ограничений: она достаточно быстро обучается, быстро генерирует рекомендации, имеет высоки уровень целевой метрики - Reacall и даёт умеренно разнообразные рекомендации, что позволит не вызвать усталости от одинаковых рекомендаций, сохраняя при этом персонализированность рекомендаций.